<u><h2>Functionalities</h2></u>
<h3>(1) Normal State</h3>
<img src='resources/basic.jpg' alt='basic-state-img'/>
<h3>(2) Obstacle State</h3>
<img src='resources/obstacle.jpg' alt='obstacle-state-img'/>
<h3>(3) Jump State</h3>
<img src='resources/up_down.jpg' alt='up_down-state-img'/>
<hr>
<u><h2>Implementations</h2></u>

In [1]:
# related modules (install opencv, pyautogui, numpy, pynput libraries if haven't been already installed)

import os  
import cv2
import datetime
import pyautogui
import numpy as np
from pynput.keyboard import Key, Controller

In [2]:
# constant variables and other status variables

OBSTACLE_BOUNDING_ORIGIN = (380, 170)
OBSTACLE_BOUNDING_END = (463, 228)
HIGHEST_JUMP_BOUNDING_ORIGIN = (258, 14)
HIGHEST_JUMP_BOUNDING_END = (341, 72)
KEYBOARD = Controller()
BOUND_AREA_COLORS = {
    1: (0, 0, 255),
    0: (0, 255, 0)
}
UP_DOWN_STATUS = {
    'up': 1,
    'down': 0
}
STATUS_IMAGE_Y = (410, 682)
STATUS_IMAGE_X = (0, 1352)
STATUS_IMAGE_RES = (1352, 330)

jumps = 0
prev_up_down_status = UP_DOWN_STATUS['down']

In [3]:
# check about the availability of contours of given area threshold value on an image

def check_contours_of_expected_area(image, area_thresh=0):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edged = cv2.Canny(gray, 100, 250)
    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    for c in contours:
        area = cv2.contourArea(c)
        if area > area_thresh:
            return 1
    return 0

In [4]:
# increase number of jumps by considering contour availability

def count_jumps(cont_avb):
    global prev_up_down_status, jumps
    
    if cont_avb is UP_DOWN_STATUS['up']:
        if cont_avb is not prev_up_down_status:
            jumps += 1
            prev_up_down_status = cont_avb
    else:
        prev_up_down_status = 0

In [10]:
# application entry

def App():
    global jumps
    
    cv2.waitKey(5000)
    
    jumps = 0
    
    KEYBOARD.press(Key.space)
    cv2.waitKey(1)
    KEYBOARD.release(Key.space)
    
    while True:
        screen_image = np.array(pyautogui.screenshot())
        screen_image = cv2.cvtColor(screen_image, cv2.COLOR_BGR2RGB)
        screen_image = screen_image[STATUS_IMAGE_Y[0]: STATUS_IMAGE_Y[1], STATUS_IMAGE_X[0]: STATUS_IMAGE_X[1]]
        
        front_obs_block = screen_image[
            OBSTACLE_BOUNDING_ORIGIN[1]: OBSTACLE_BOUNDING_END[1] + 1, 
            OBSTACLE_BOUNDING_ORIGIN[0]: OBSTACLE_BOUNDING_END[0] + 1
        ]
        highest_jump_block = screen_image[
            HIGHEST_JUMP_BOUNDING_ORIGIN[1]: HIGHEST_JUMP_BOUNDING_END[1] + 1,
            HIGHEST_JUMP_BOUNDING_ORIGIN[0]: HIGHEST_JUMP_BOUNDING_END[0] + 1
        ]
        
        avb = check_contours_of_expected_area(front_obs_block)
        obstacle_area_color, obstacle_status = BOUND_AREA_COLORS[0], 0
        
        if avb:
            obstacle_area_color, obstacle_status = BOUND_AREA_COLORS[1], 1
                
            KEYBOARD.press(Key.space)
            cv2.waitKey(1)
            KEYBOARD.release(Key.space)
        
        avb = check_contours_of_expected_area(highest_jump_block)
        highest_jump_bound_area_color, jump_status = BOUND_AREA_COLORS[0], 0
        
        if avb:
            highest_jump_bound_area_color, jump_status = BOUND_AREA_COLORS[1], 1
        count_jumps(avb)
            
        cv2.rectangle(screen_image, OBSTACLE_BOUNDING_ORIGIN, OBSTACLE_BOUNDING_END, obstacle_area_color, 2)
        cv2.rectangle(screen_image, HIGHEST_JUMP_BOUNDING_ORIGIN, HIGHEST_JUMP_BOUNDING_END, highest_jump_bound_area_color, 2)
        cv2.rectangle(screen_image, (OBSTACLE_BOUNDING_ORIGIN[0], OBSTACLE_BOUNDING_ORIGIN[1] - 22), (OBSTACLE_BOUNDING_ORIGIN[0] + 22, OBSTACLE_BOUNDING_ORIGIN[1]), obstacle_area_color, -1)
        cv2.rectangle(screen_image, (HIGHEST_JUMP_BOUNDING_END[0], HIGHEST_JUMP_BOUNDING_ORIGIN[1]), (HIGHEST_JUMP_BOUNDING_END[0] + 22, HIGHEST_JUMP_BOUNDING_ORIGIN[1] + 22), highest_jump_bound_area_color, -1)
        
        screen_image = cv2.putText(screen_image, 'JUMPS: {}'.format(jumps), (570, 43), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)
        screen_image = cv2.putText(screen_image, '{}'.format(obstacle_status), (384, 167), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        screen_image = cv2.putText(screen_image, '{}'.format(jump_status), (345, 32), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        cv2.imshow("LIVE | Automated Chrome Dinosaur", screen_image)
        
        k = cv2.waitKey(1)
        if k == 27: 
            return                                                                                   

In [11]:
App()               
cv2.destroyAllWindows()